# [Module 8] Recommendation API 호출 

# 0. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [4]:
%store -r

# 1. 아이템 테이터 로딩

- 추천 결과의 ITEM_ID 의 상세 내용을 확인하기 위해서 로딩 함.

In [5]:
from src.p_utils import get_item_list_details

items_df = pd.read_csv(base_items_filename)
items_cvr_df = pd.read_csv(cvr_items_filename)

items_df.head()

,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,Sans Pareil Scarf,apparel,scarf,Sans pareil scarf for women,124.99
1,cfafd627-7d6b-43a5-be05-4c7937be417d,Chef Knife,housewares,kitchen,A must-have for your kitchen,57.99
2,6e6ad102-7510-4a02-b8ce-5a0cd6f431d1,Gainsboro Jacket,apparel,jacket,This gainsboro jacket for women is perfect for...,133.99
3,49b89871-5fe7-4898-b99d-953e15fb42b2,High Definition Speakers,electronics,speaker,High definition speakers to fill the house wit...,196.99
4,5cb18925-3a3c-4867-8f1c-46efd7eba067,Spiffy Sandals,footwear,sandals,This spiffy pair of sandals for woman is perfe...,9.99


In [6]:
items_cvr_df.head()

,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,00096972-5f6b-44df-917b-f7d21ae5644c,0.04106,Pink Shirt,apparel,shirt,Swanky dress for women,225.99
1,0016fde3-0910-4cc1-8ef6-90e15f271073,0.01406,Farmed Salmon For Sushi,groceries,seafood,"Flavorful farmed salmon, always sourced sustai...",24.99
2,00225258-dbfb-4103-a573-007386571a49,0.06765,Easter Decorative Egg,seasonal,easter,A must-have for April,16.99
3,003e4953-d6cb-400c-90f6-9b0216b4603e,0.03637,Drought-Resistant Indoor Plant,floral,plant,Drought-resistant indoor plant grown sustainab...,76.99
4,004112e9-dca1-4402-ae6d-74e2b80b8c05,0.03481,Mango Coat,apparel,jacket,Mango coat for men about town,184.99


# 2. Recommendation API 호출

- 두 명의 user_id (에: 1, 100) 에 대해서 추천 결과를 얻음.

### Base 솔루션 vs CVR 가중치 솔루션 추천 : user_id = 1

In [7]:
user_id = 1

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = base_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")    
print("Base - Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  1
-------------------
Base - Solution Recommendation


,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,95677239-43cc-44e4-a97f-4a9c072210ae,Gainsboro Armchair,furniture,chairs,Swanky gainsboro armchair for any room,211.99
1,7f4248fb-69ed-46d5-b038-9502b662ad13,Tan Accent Table,furniture,tables,Funky tan accent table to enhance your room,181.99
2,2a49853d-f818-4a46-8008-6ee9ef256809,Dark Salmon Work Table,furniture,tables,Increase your productivity with this incompara...,238.99
3,d3d53458-ea05-4544-9a11-5a44df873e4d,White Worktable,furniture,tables,Increase your productivity with this definitiv...,160.99
4,68f56b2c-5807-42ee-9436-12257fb5bcfb,Glass Dining Table,furniture,tables,This unsurpassed glass dining table will delig...,337.99
5,06c5b78d-d957-411c-902c-ab0d0110cca0,Dark Olive Green Dining Table,furniture,tables,Unsurpassed dark olive green dining room for t...,200.99
6,4f566f59-4400-437a-803e-88f81b303eec,Rosy-Brown Dining Table,furniture,tables,This definitive rosy-brown dining table will d...,183.99
7,b874dde3-1102-46a9-8a6c-28981ac151a8,Gainsboro Accent Table,furniture,tables,Fashionable gainsboro accent table to enhance ...,349.99
8,489dee3d-6abf-4470-bb94-8b9c0d12f18e,Saddle Brown Office Chair,furniture,chairs,Increase your productivity with this nice sadd...,210.99
9,448bac96-2ade-4410-9632-e208c456fcf7,Powder Blue Work Table,furniture,tables,Increase your productivity with this unrivaled...,217.99


In [8]:
user_id = 1

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = cvr_m_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")  
print("CVR Medium - Solution Recommendation")
get_item_list_details(items_cvr_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  1
-------------------
CVR Medium - Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,7f4248fb-69ed-46d5-b038-9502b662ad13,0.08209,Tan Accent Table,furniture,tables,Funky tan accent table to enhance your room,181.99
1,4daf5ac5-2aa9-42df-b6da-895ba09e9012,0.08141,Dark Slate Gray Sofa,furniture,sofas,Enjoy this nice dark slate gray sofa,764.99
2,47636cd1-731c-47d1-8d0b-612575af0a5c,0.08783,Leather Armchair,furniture,chairs,Modish leather armchair for any room,198.99
3,7673321f-6042-4ad7-bc1d-4471ee1dcd1b,0.02802,Sienna Sofa,furniture,sofas,This upscale sienna sofa will be the highlight...,790.99
4,95677239-43cc-44e4-a97f-4a9c072210ae,0.08251,Gainsboro Armchair,furniture,chairs,Swanky gainsboro armchair for any room,211.99
5,c02e9ebe-eb65-4a03-9078-e4faf330ddd0,0.07470,Wheat Hue Work Table,furniture,tables,Increase your productivity with this unparalle...,320.99
6,da1fb708-edc1-4180-b9f0-1a6fb1fd3416,0.08042,Light Gray Worktable,furniture,tables,Definitive light gray worktable for your office,245.99
7,3ce61cd6-8cbd-42d7-9563-37eb7378fbc5,0.06062,Saddle Brown Office Chair,furniture,chairs,Outstanding saddle brown chair for your office,151.99
8,b842b868-4117-4254-aa1c-be43b05abb87,0.07982,Dark Khaki Office Chair,furniture,chairs,Incomparable dark khaki chair for your office,124.99
9,1f88855c-7ed3-4796-9143-dcdd395c03ad,0.01387,Dark Gray Armchair,furniture,chairs,Cool dark gray armchair for any room,189.99


In [9]:
user_id = 1

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = cvr_h_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")  
print("CVR High - Solution Recommendation")
get_item_list_details(items_cvr_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  1
-------------------
CVR High - Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,47636cd1-731c-47d1-8d0b-612575af0a5c,0.08783,Leather Armchair,furniture,chairs,Modish leather armchair for any room,198.99
1,3293c333-10c8-4837-a07d-d8cd9e0f9bf8,0.12314,Saddle Brown Dresser,furniture,dressers,This unequaled dresser has lots of drawers to ...,120.99
2,d055f31f-2e3e-4338-ac64-521f88779b50,0.16342,Dark Gray Worktable,furniture,tables,Unsurpassed dark gray worktable for your office,178.99
3,1ddc59b3-7f01-4039-8f07-21a30362ab75,0.09425,Black Office Chair,furniture,chairs,Increase your productivity with this soft blac...,177.99
4,dfe8e597-14f3-48af-b86b-cfd80f2321cf,0.16175,Wooden Worktable,furniture,tables,Increase your productivity with this incompara...,321.99
5,7f4248fb-69ed-46d5-b038-9502b662ad13,0.08209,Tan Accent Table,furniture,tables,Funky tan accent table to enhance your room,181.99
6,3d7fefba-b18a-4571-8a5f-0fc7547c7513,0.14215,Wall Clock,homedecor,clock,This wall clock will keep you on time,144.99
7,7719893d-b5d8-4277-ac66-b2fb3394063e,0.11550,White Work Table,furniture,tables,Increase your productivity with this unsurpass...,153.99
8,53b930e1-a245-4a24-8d5b-5343f6892b37,0.19897,Frame,homedecor,decorative,This frame is a must-have for your home decor,56.99
9,75917ca3-d96d-49c9-b99a-60baac872b7a,0.12320,Rosy-Brown Coffee Table,furniture,tables,Supercool rosy-brown coffee table to enliven y...,281.99


### Base 솔루션 vs CVR 가중치 솔루션 추천 : user_id = 100

In [10]:
user_id = 100

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = base_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")    
print("Base - Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  100
-------------------
Base - Solution Recommendation


,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,387f55e6-0eb9-47d6-9ef1-56a80b884430,Electric Bass,instruments,strings,This electric bass will delight the most deman...,57.99
1,6ca2a410-aba9-4c73-9dae-d188412e0e50,Electric Guitar,instruments,strings,This electric guitar will delight the most dem...,114.99
2,76fa669b-1611-4f31-8377-55c3e701ced4,Saxophone,instruments,wind,This saxophone is perfect for amateur and prof...,95.99
3,66725b05-5d69-48f5-bafc-883957a493d1,French Horn,instruments,wind,This french horn is perfect for amateur and pr...,192.99
4,e09d3267-1a87-4518-9fa7-fa9ed847c80d,Accurate Microphone,instruments,microphone,This accurate microphone can be relied upon in...,499.99
5,a54fbc82-c85d-4d21-b907-e8cdbbff69d2,Acoustic Bass,instruments,strings,This acoustic bass will delight the most deman...,221.99
6,fd34b35b-de76-496e-89f1-6ef65969e1a4,Visit Mexico,books,travel,Plan your next trip with this essential refere...,15.99
7,39fe5f7d-c84a-4492-a04b-5b673a1682d6,High Definition Microphone,instruments,microphone,This high definition microphone can be relied ...,151.99
8,47707417-48c4-4faa-84ee-a06414d29e46,Visit Spain,books,travel,Plan your next trip with this essential refere...,9.99
9,ac099a60-9187-4d4f-97b4-6bdfb14ba521,Visit Iceland,books,travel,Plan your next trip with this essential refere...,11.99


In [11]:
user_id = 100

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = cvr_m_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")  
print("CVR Medium - Solution Recommendation")
get_item_list_details(items_cvr_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  100
-------------------
CVR Medium - Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,76fa669b-1611-4f31-8377-55c3e701ced4,0.13318,Saxophone,instruments,wind,This saxophone is perfect for amateur and prof...,95.99
1,e09d3267-1a87-4518-9fa7-fa9ed847c80d,0.05859,Accurate Microphone,instruments,microphone,This accurate microphone can be relied upon in...,499.99
2,6ca2a410-aba9-4c73-9dae-d188412e0e50,0.00508,Electric Guitar,instruments,strings,This electric guitar will delight the most dem...,114.99
3,387f55e6-0eb9-47d6-9ef1-56a80b884430,0.00592,Electric Bass,instruments,strings,This electric bass will delight the most deman...,57.99
4,66725b05-5d69-48f5-bafc-883957a493d1,0.01665,French Horn,instruments,wind,This french horn is perfect for amateur and pr...,192.99
5,b803aff7-6e4b-4beb-8d1d-dc7fe609274d,0.08815,Laptop Computer,electronics,computer,Laptop computer for your computing needs,9.99
6,25ce5128-8f09-4d01-8201-91198fdb9d3e,0.15034,Television,electronics,television,This LED television delivers a sharp picture,484.99
7,39b4eae7-37ee-4e6d-a617-afdf77129be5,0.08566,Clarinet,instruments,wind,This clarinet is perfect for amateur and profe...,192.99
8,5dd5e235-4d2e-432b-a1b9-8e738dc8aa3a,0.14509,Desktop Computer,electronics,computer,Desktop computer for your computing needs,9.99
9,8fb0eebc-972c-4934-ad7d-1405c2d41640,0.20330,Faultless Headphones,electronics,headphones,This faultless pair of headphones is ideal in ...,120.99


In [12]:
user_id = 100

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = cvr_h_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")  
print("CVR High - Solution Recommendation")
get_item_list_details(items_cvr_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  100
-------------------
CVR High - Solution Recommendation


,ITEM_ID,CVR,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,76fa669b-1611-4f31-8377-55c3e701ced4,0.13318,Saxophone,instruments,wind,This saxophone is perfect for amateur and prof...,95.99
1,e09d3267-1a87-4518-9fa7-fa9ed847c80d,0.05859,Accurate Microphone,instruments,microphone,This accurate microphone can be relied upon in...,499.99
2,10a400a4-3afa-4b7b-989e-dc7ed6298efd,0.07400,Drum Set,instruments,percussion,This drum set will delight the most demanding ...,364.99
3,9ffda50b-e5a9-47ae-9d63-a0ab90f8cbac,0.08450,High Fidelity Microphone,instruments,microphone,This high fidelity microphone can be relied up...,253.99
4,b1b9c461-26c2-4769-b017-0791b467acf3,0.13053,Grand Piano,instruments,keys,This grand piano will delight the most demandi...,146.99
5,6ca2a410-aba9-4c73-9dae-d188412e0e50,0.00508,Electric Guitar,instruments,strings,This electric guitar will delight the most dem...,114.99
6,8fb0eebc-972c-4934-ad7d-1405c2d41640,0.20330,Faultless Headphones,electronics,headphones,This faultless pair of headphones is ideal in ...,120.99
7,7d82f4d2-d9b7-428f-b19c-f8e3c7d45d2f,0.09611,Upright Piano,instruments,keys,This upright piano is perfect for amateur and ...,239.99
8,28cc6e51-6e20-4aec-9751-572e19134b6d,0.16181,Acoustic Drum,instruments,percussion,This acoustic drum will delight the most deman...,267.99
9,b30f5517-9cc2-45ef-87d8-5bb60a443c36,0.09802,Video Camera,electronics,camera,This video camera is perfect for capturing tho...,752.99
